# ScalaPy tutorial
In this notebook I will show how to use Scalapy -- a library for using python from Scala.

Important Links:
- Main Site: [https://scalapy.dev/](https://scalapy.dev/)
- Github: [https://github.com/scalapy/scalapy](https://github.com/scalapy/scalapy)
- Facade Examples
    - [OpenAI Gym](https://github.com/cric96/scalapy-gym)
    - [Tensorflow](https://github.com/shadaj/scalapy-tensorflow)
    - [NumPy](https://github.com/shadaj/scalapy-numpy)
 
 ScalaPy is released for Scala 2.11 - 3 under the organization `dev.scalapy` on Maven Central, the last version is `0.5.3`.


In [1]:
import $ivy.`dev.scalapy::scalapy-core:0.5.3`

import $ivy.$                                


## Overview
Scalapy serves as a bridge, enabling seamless invocation of Python functions directly from Scala code. 

This library harnesses the scala.Dynamic trait to provide dynamic typing functionalities akin to those found in scala.js, thereby streamlining the integration process. The scala macro system performs the majority of the type conversions.

At present, the library is compatible with Python versions 3.6 through 3.10. While an initial working version exists for scala 3.0, it lacks comprehensive support for numerous facade definition features, such as curried methods.

Consequently, for the purposes of this tutorial, we will be utilizing scala version 2.13.

## Main Concepts
ScalaPy is structured upon several key abstractions, each tailored to facilitate seamless interaction between Scala and Python. These include:

- `py.Any`: This is a Scala type designed to encapsulate any Python object. It serves as the foundational type from which any facade you create should inherit.
- `py.Dynamic`: Another Scala type, `py.Dynamic` has the capability to hold a Python object and allows for the invocation of Python methods in an unsafe manner. This is particularly useful for manipulating Python objects within Scala code.
- `py.Module`: This represents a Python module in Scala. It is the go-to type for accessing Python functions and classes, serving as a bridge to Python’s modular architecture.
- `py.global`: As a specialized instance of `py.Module`, `py.global` mirrors the global namespace in Python. It provides a pathway to Python’s global functions and facilitates their accessibility within Scala.
- `py.readwrite.Writer` and `py.readwrite.Reader`: These are typeclasses that act as translators between Scala and Python objects. They allow for the conversion of Scala objects to Python objects and vice versa, thus enabling a smooth bidirectional flow of data.

These core concepts are fundamental to ScalaPy's design, ensuring that Scala developers can interact with Python's rich ecosystem in a type-safe and intuitive manner.


## Walkthrough
In this section, we will walk through the basic steps required to use ScalaPy.

In [2]:
// Entrypoint
import me.shadaj.scalapy.py

import me.shadaj.scalapy.py


In [3]:
// Access to global Python function 
py.Dynamic.global.int("100")

res2: Dynamic = 100

### Sequence conversion
ScalaPy provides a number of implicit conversions to facilitate the conversion of Scala sequences to Python sequences and vice versa. These conversions are defined in the `py.SeqConverters` package.
There are mainly two types of conversions:
- copy: This conversion creates a copy of the original sequence. Any changes made to the copy will not affect the original sequence.
- proxy: This conversion creates a proxy of the original sequence. Any changes made to the proxy will affect the original sequence.


In [4]:
// Immutable copy
import me.shadaj.scalapy.py.SeqConverters // Conversion for Scala-Python list 
// Convert scala list to python list
val elements = 10 :: 20 :: 30 :: Nil
val pythonList = elements.toPythonCopy

import me.shadaj.scalapy.py.SeqConverters // Conversion for Scala-Python list 
// Convert scala list to python list

elements: List[Int] = List(10, 20, 30)
pythonList: Any = [10, 20, 30]

### On python Dynamic
You Use python list as Dynamic
since it is possible to convert any python object to other object if a Reader exists in the context.
Any `py.Any` can be converted `to py.Dynamic`

In [5]:
val unsafeList = pythonList.as[py.Dynamic]
unsafeList.bracketUpdate(0, 100)

unsafeList: Dynamic = [100, 20, 30]

In [6]:
// The original sequence is not affected
elements

res5: List[Int] = List(10, 20, 30)

In [32]:
val mutableList = scala.collection.mutable.Seq[Int](1, 2, 3)
// Proxy to the original object, the changes mutableList will be reflected
val proxy = mutableList.toPythonProxy.as[py.Dynamic]

mutableList: collection.mutable.Seq[Int] = ArrayBuffer(1, 2, 3)
proxy: py.Dynamic = <SequenceProxy object at 0x7f290192cfd0>

In [35]:
mutableList(0) = 100 // updating the original object change the value in the proxy
proxy.bracketAccess(0)

res35_1: py.Dynamic = 100

In [36]:
// It is possible to convert back a python list to scala list
pythonList.as[Seq[Int]]

res36: Seq[Int] = List(100, 100, 30)

In [38]:
// Python list can be converted also to scala mutable list
// In this way, it is possible to update the original python list
val mutablePythonList = pythonList.as[scala.collection.mutable.Seq[Int]]
mutablePythonList(2) = 100
pythonList 

mutablePythonList: collection.mutable.Seq[Int] = Seq(100, 100, 100)
res38_2: py.Any = [100, 100, 100]

### On PyQuote

Occasionally, accessing specific Python functions directly within Scala is required. 
In such scenarios, quoting code is a practical approach. 
This method allows you to write Python code directly in Scala and interpolate Python objects using the `$` symbol. 
This integration technique enables seamless interaction between Scala and Python, facilitating the use of Python's extensive libraries and functions within a Scala environment.


In [39]:

import py.PyQuote
py"list(map(lambda x: x + 1, $pythonList))"

import py.PyQuote
res39_1: py.Dynamic = [101, 101, 101]

## Modules 
As mentioned earlier, `py.Module` is the primary type for accessing Python functions and classes. It is the ScalaPy equivalent of a Python module.
There are two kinds of modules:
- Dynamic/Unsafe: This module is created by calling `py.module("module")`, and it returns a `py.Dynamic` object. This module is unsafe because it does not check if the module exists or not.
- StaticModule: This module is mainly a typed facade for a Python module. It is created by extending `py.StaticModule` and defining the methods and attributes of the module.

In [40]:
// Use python module
val os = py.module("os")

os: py.Module = <module 'os' from '/usr/lib/python3.8/os.py'>

In [41]:
os.getcwd()

res41: py.Dynamic = /home

## Safe Module

In [44]:
@py.native // necessary to mark the trait as a native python module
object os extends py.StaticModule("os") {
  // It converts the python object to String because in the context exists a Reader[String]
  def getcwd(): String = py.native
}

defined object os

In [45]:
os.getcwd()

res45: String = "/home"

## Typed Facade
It is also possible to create a typed facade for any Python class. 
This is done by extending py.Object and defining the methods and attributes of the class.
The facade can then be used using as[FacadeType] on a py.Any object.

In [46]:
@py.native trait PyString extends py.Object {
  def count(subsequence: String): Int = py.native
}

defined trait PyString

In [47]:
val string = py.module("string").digits.as[PyString]

string: PyString = 0123456789

In [48]:
string.count("hello1")

res48: Int = 0

### Advanced Facade
It is also possible to create a facade using generic programming, type bounds, and type classes.

In [49]:
import py.PyBracketAccess
import me.shadaj.scalapy.readwrite.{Reader, Writer}
// A complex example, it is possible to use generics in facade only if http://127.0.0.1:8888/lab?token=73dda80fbe123ad9d75007ca98900c061a27233772c0a437Reader and Writer exist.
@py.native
trait SafePythonList[A] extends py.Object {
  @PyBracketAccess
  def apply(index: Int)(implicit reader: Reader[A]): A = py.native

  @PyBracketAccess
  def update(index: A, newValue: Int)(implicit writer: Writer[A]): Unit = py.native
}

import py.PyBracketAccess
import me.shadaj.scalapy.readwrite.{Reader, Writer}
defined trait SafePythonList

In [50]:
val safeList = py"[1, 2, 3]".as[SafePythonList[Int]]
safeList(0)
safeList(0) = 10
// Illegal -> safeList(0) = "10"

safeList: SafePythonList[Int] = [10, 2, 3]
res50_1: Int = 1